In [1]:
import os
import sys
sys.path.append("..")
import phage_init

In [2]:
import numpy
import itertools
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from scipy import stats
from Bio.Alphabet import IUPAC
#from itertools import permutations


In [ ]:
#from numba import jit,vectorize

In [3]:
all_fasta=(os.path.join(phage_init.fasta_dir_2,"major_capsid_all_clustered.fasta"),os.path.join(phage_init.fasta_dir_2,"minor_capsid_all_clustered.fasta"),
           os.path.join(phage_init.fasta_dir_2,"baseplate_all_clustered.fasta"),os.path.join(phage_init.fasta_dir_2,"major_tail_all_clustered.fasta"),
           os.path.join(phage_init.fasta_dir_2,"minor_tail_all_clustered.fasta"),os.path.join(phage_init.fasta_dir_2,"portal_all_clustered.fasta"),
           os.path.join(phage_init.fasta_dir_2,"tail_fiber_all_clustered.fasta"),os.path.join(phage_init.fasta_dir_2,"shaft_all_clustered.fasta"),
           os.path.join(phage_init.fasta_dir_2,"collar_all_clustered.fasta"),os.path.join(phage_init.fasta_dir_2,"HTJ_all_clustered.fasta"))

In [11]:

def extract_all(fasta_list):
#    AA=["A","C","D","E","F","G","H","I","K","L","M","N","P","Q","R","S","T","V","W","Y"]
    SC=["1","2","3","4","5","6","7"]
#    di_pep = [''.join(i) for i in itertools.product(AA, repeat = 2)]
#    tri_pep = [''.join(i) for i in itertools.product(AA, repeat = 3)]
#    di_sc = [''.join(i) for i in itertools.product(SC, repeat = 2)]
#    tri_sc = [''.join(i) for i in itertools.product(SC, repeat = 3)]
    tetra_sc = [''.join(i) for i in itertools.product(SC, repeat = 4)]
    prot_class=0;
    myseq="AILMVNQSTGPCHKPDEFWY"
    trantab2=myseq.maketrans("AILMVNQSTGPCHKPDEFWY","11111222233455566777")
    arr = numpy.empty((0,2409), dtype=numpy.float)
    class_arr = numpy.empty((0), dtype=numpy.int)
    for file in fasta_list:
        print('####################' + file)
        for record in SeqIO.parse(file, "fasta"):
            ll=len(record.seq)
            seqq=record.seq.__str__().upper()
            seqqq=seqq.replace('X','A').replace('J','L').replace('*','A').replace('Z','E').replace('B','D')
            X = ProteinAnalysis(seqqq)
            tt= [X.isoelectric_point(), X.instability_index(),ll,X.aromaticity(),
                X.molar_extinction_coefficient()[0],X.molar_extinction_coefficient()[1],
                X.gravy(),X.molecular_weight()]
            tt_n = numpy.asarray(tt,dtype=numpy.float)
            myseq=seqq.translate(trantab2)
            
 #           di_pep_count=[seqq.count(i)/(ll-1) for i in di_pep]
 #           di_pep_count_n = numpy.asarray(di_pep_count,dtype=numpy.float)
 #           
 #           tri_pep_count=[seqq.count(i)/(ll-2) for i in tri_pep]
 #           tri_pep_count_n = numpy.asarray(tri_pep_count,dtype=numpy.float)
            
  #          di_sc_count=[myseq.count(i)/(ll-1) for i in di_sc]
  #          di_sc_count_n = numpy.asarray(di_sc_count,dtype=numpy.float)
 #   
 #           tri_sc_count=[myseq.count(i)/(ll-2) for i in tri_sc]
 #           tri_sc_count_n = numpy.asarray(tri_sc_count,dtype=numpy.float)
            tetra_sc_count=[myseq.count(i)/(ll-3) for i in tetra_sc]
            tetra_sc_count_n = numpy.asarray(tetra_sc_count,dtype=numpy.float)
            cat_n= numpy.concatenate((tetra_sc_count_n,tt_n))
    #        cat_n= numpy.concatenate((di_pep_count_n,tri_pep_count_n,di_sc_count_n,tri_sc_count_n,tt_n))
    #        print(cat_n.shape)
            cat_n = cat_n.reshape((1,cat_n.shape[0]))

            arr = numpy.append(arr,cat_n , axis=0)
            class_arr = numpy.append(class_arr,prot_class)
        prot_class+=1
    return (arr,class_arr)
    

In [ ]:
(arr,class_arr)=extract_all(all_fasta)
print(arr.shape)

####################/home/adrian/git-presentations/keras/phage_protein/download_seq/all/clustered/major_capsid_all_clustered.fasta
####################/home/adrian/git-presentations/keras/phage_protein/download_seq/all/clustered/minor_capsid_all_clustered.fasta
####################/home/adrian/git-presentations/keras/phage_protein/download_seq/all/clustered/baseplate_all_clustered.fasta
####################/home/adrian/git-presentations/keras/phage_protein/download_seq/all/clustered/major_tail_all_clustered.fasta


In [ ]:

arr_z=numpy.apply_along_axis(stats.zscore,0,arr)



In [ ]:
nb_classes = 10
one_hot_targets = numpy.eye(nb_classes)[class_arr]

In [ ]:
final = numpy.concatenate((arr_z, one_hot_targets), axis=1)

In [ ]:
mean_arr=numpy.apply_along_axis(numpy.mean,0,arr)
#print(mean_arr)
print(mean_arr.shape)
std_arr=numpy.apply_along_axis(numpy.std,0,arr)
print(std_arr.shape)

In [ ]:
numpy.random.shuffle(final)

In [ ]:
import pickle
##number of proteins for training
tt=50000
##number of features
f_num=arr.shape[1]
pickle.dump(final[0:tt,0:f_num], open( os.path.join(phage_init.data_dir,"train_x_data2_tetra_sc_p.p"), "wb" ) )
pickle.dump(final[tt:,0:f_num],  open( os.path.join(phage_init.data_dir,"test_x_data2_tetra_sc_p.p") , "wb" ) )
pickle.dump(final[0:tt,f_num:], open( os.path.join(phage_init.data_dir,"train_y_data2_tetra_sc_p.p"), "wb" ) )
pickle.dump(final[tt:,f_num:],  open( os.path.join(phage_init.data_dir,"test_y_data2_tetra_sc_p.p") , "wb" ) )
pickle.dump(mean_arr,  open( os.path.join(phage_init.data_dir,"mean_data2_tetra_sc_p.p") , "wb" ) )
pickle.dump(std_arr,  open( os.path.join(phage_init.data_dir,"std_data2_tetra_sc_p.p") , "wb" ) )